In [786]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, Lasso, Ridge, RidgeCV, ElasticNet
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [787]:
# Input data files are available in the "/kaggle/input" directory


# Import the file
df = pd.read_csv('lisbon-houses.csv.xls') 

# print the first lines
df.head(20)


,Id,Condition,PropertyType,PropertySubType,Bedrooms,Bathrooms,AreaNet,AreaGross,Parking,Latitude,Longitude,Country,District,Municipality,Parish,Price M2,Price
0,101,Used,Homes,Apartment,3,1,76,152,0,38.7792,-9.1186,Portugal,Lisboa,Lisboa,Olivais,2463,198000
1,102,Used,Homes,Duplex,5,3,190,380,0,38.7056,-9.1784,Portugal,Lisboa,Lisboa,Alcantara,3125,1270000
2,103,Used,Homes,Apartment,1,1,26,52,0,38.7058,-9.1639,Portugal,Lisboa,Lisboa,Estrela,4005,140000
3,104,Used,Homes,Apartment,5,4,185,370,0,38.7466,-9.1640,Portugal,Lisboa,Lisboa,Sao Domingos de Benfica,3412,995000
4,105,Used,Homes,Apartment,7,1,150,300,0,38.7323,-9.1287,Portugal,Lisboa,Lisboa,Arroios,3277,570000
5,106,Used,Homes,Apartment,3,2,95,190,0,38.6965,-9.2099,Portugal,Lisboa,Lisboa,Belem,3542,425000
6,107,Used,Homes,Apartment,3,1,76,152,0,38.7544,-9.1129,Portugal,Lisboa,Lisboa,Marvila,2881,130000
7,108,Used,Homes,Apartment,3,1,81,162,0,38.7561,-9.1796,Portugal,Lisboa,Lisboa,Sao Domingos de Benfica,3412,215000
8,109,Used,Homes,Apartment,2,1,58,116,0,38.7792,-9.1186,Portugal,Lisboa,Lisboa,Olivais,2463,196000
9,110,Used,Homes,Apartment,5,3,286,572,0,38.7632,-9.1687,Portugal,Lisboa,Lisboa,Lumiar,2927,925000


In [788]:
df = df.drop(['Country', 'District','Municipality', 'Id', 'Condition', 'PropertyType', 'PropertySubType', 'Parish' ], axis=1)
df.head(5)

,Bedrooms,Bathrooms,AreaNet,AreaGross,Parking,Latitude,Longitude,Price M2,Price
0,3,1,76,152,0,38.7792,-9.1186,2463,198000
1,5,3,190,380,0,38.7056,-9.1784,3125,1270000
2,1,1,26,52,0,38.7058,-9.1639,4005,140000
3,5,4,185,370,0,38.7466,-9.1640,3412,995000
4,7,1,150,300,0,38.7323,-9.1287,3277,570000


In [789]:
#df=pd.get_dummies(df)
#df.head()

In [790]:
y_min = df['Price'].min()
y_max = df['Price'].max()

def original_value_price(pred):          
    return pred*(y_max - y_min) + y_min

def normalize_value(val):
    return (val - val.min()) / (val.max() - val.min())

# transform numeric variables between 0 and 1.
df['Bedrooms'] =  normalize_value(df['Bedrooms']) 
df['Bathrooms'] = normalize_value(df['Bathrooms']) 
df['AreaNet'] = normalize_value(df['AreaNet']) 
df['AreaGross'] = normalize_value(df['AreaGross']) 
df['Price M2'] = normalize_value(df['Price M2']) 
df['Price'] = normalize_value(df['Price'])
df['Latitude'] = normalize_value(df['Latitude'])
df['Longitude'] = normalize_value(df['Longitude'])


df.head(15)

,Bedrooms,Bathrooms,AreaNet,AreaGross,Parking,Latitude,Longitude,Price M2,Price
0,0.272727,0.166667,0.096364,0.096364,0,0.885684,0.810107,0.023753,0.027798
1,0.454545,0.500000,0.303636,0.303636,0,0.099359,0.352221,0.248388,0.291513
2,0.090909,0.166667,0.005455,0.005455,0,0.101496,0.463247,0.546997,0.013530
3,0.454545,0.666667,0.294545,0.294545,0,0.537393,0.462481,0.345775,0.223862
4,0.636364,0.166667,0.230909,0.230909,0,0.384615,0.732772,0.299966,0.119311
5,0.272727,0.333333,0.130909,0.130909,0,0.002137,0.111026,0.389888,0.083641
6,0.272727,0.166667,0.096364,0.096364,0,0.620726,0.853752,0.165592,0.011070
7,0.272727,0.166667,0.105455,0.105455,0,0.638889,0.343032,0.345775,0.031980
8,0.181818,0.166667,0.063636,0.063636,0,0.885684,0.810107,0.023753,0.027306
9,0.454545,0.500000,0.478182,0.478182,0,0.714744,0.426493,0.181201,0.206642


In [791]:
#df = df.drop(['Longitude', 'Latitude'], axis=1)
#df.head(16)

In [792]:
y = df["Price"]
X = df.drop("Price", axis=1)

X_arr = X.values
y_arr = y.values

print('X_arr shape: ', X_arr.shape)
print('y_arr shape: ', y_arr.shape)

X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size = 0.10, shuffle = True, random_state=0)

print(X_test.shape)


X_arr shape:  (246, 8)
y_arr shape:  (246,)
(25, 8)


In [793]:
#We made a linear regression so that we can see witch points will be the outliers
model_lr = LinearRegression()
model_lr.fit(X_train,y_train)
y_p=model_lr.predict(X_train)

error=y_train-y_p

#After we obtain the error matrix, we will find the 2 point with worst error
ind = np.argsort(np.abs(error))[-2:]
print(ind)

#Values of X_train and Y_train without the outliers
newY=np.delete(y_train,ind)
newX=np.delete(X_train,ind, axis=0)


#Finding the best alpha for the model 

model_lasso2 = Lasso(alpha = 0.001)
model_lasso2.fit(newX,newY) 

y_pred = model_lasso2.predict(X_test)

error = y_pred - y_test
#print(error)
squared_error = np.dot(error, error)
mean_squared_error = squared_error / y_test.shape[0]
print(np.sqrt(mean_squared_error))

[88 27]
0.030505960229744507


In [794]:
#LASSO
#modelo_lassoCV = LassoCV(cv=10, fit_intercept=True,tol=0.001,eps=0.0000001,n_alphas=95).fit(X_train, y_train)
#model_lasso = Lasso(alpha = modelo_lassoCV.alpha_,tol=0.001,fit_intercept=True)
model_lasso = Lasso(alpha = 0.000001,tol=0.00001,fit_intercept=True)
model_lasso.fit(X_train,y_train)
y_pred = model_lasso.predict(X_test)

error = y_pred - y_test
print(error)
squared_error = np.dot(error, error)
mean_squared_error = squared_error / y_test.shape[0]
print(np.sqrt(mean_squared_error))

[-0.00697118 -0.03396545  0.00530014 -0.03396545 -0.0076279   0.07008289
  0.0132472   0.05680886  0.0353706  -0.00218926  0.03563017  0.04342905
  0.01049224  0.0069434  -0.05044849  0.00938568 -0.02359686  0.02775634
 -0.00219486  0.01251614 -0.00830041  0.0237037   0.01836657 -0.01306458
 -0.00737291]
0.02873413323047799


In [795]:
model_lr = LinearRegression()
model_lr.fit(X_train,y_train)
y_p=model_lr.predict(X_test)

error = y_p - y_test
print(error)
squared_error = np.dot(error, error)
mean_squared_error = squared_error / y_test.shape[0]
print(np.sqrt(mean_squared_error))


[-0.0069705  -0.03394791  0.00530209 -0.03394791 -0.00761793  0.07012449
  0.01328145  0.05684601  0.0353796  -0.00224126  0.03563547  0.04343195
  0.010541    0.00692632 -0.05039812  0.00939344 -0.0236542   0.02775519
 -0.00219527  0.01251202 -0.00831799  0.0236794   0.01838767 -0.01303515
 -0.00738492]
0.02873946682462949


In [796]:
print(y_p)

print("____")

print(X_test[13])

[0.17138128 0.06814311 0.13150135 0.06814311 0.06126276 0.10333483
 0.2186935  0.1958374  0.11287038 0.17119048 0.08114593 0.14306295
 0.1802827  0.03853764 0.22389462 0.04014374 0.07351677 0.02775519
 0.19632872 0.11829308 0.02858238 0.08025997 0.1610691  0.13825636
 0.04427559]
____
[0.18181818 0.16666667 0.06909091 0.06909091 0.         0.32264957
 0.81699847 0.19681032]


In [797]:
x_try  = np.array([ 0.18181818, 0.16666667, 0.06909091, 0.06909091, 0, 0.32264957 ,0.81699847, 0.19681032]).reshape(1,-1)
print(x_try.shape)
#print(x_try.shape)
#print(X_test.shape)
previst_P = model_lr.predict(x_try)
Price = original_value_price(previst_P)
print(previst_P)
print(Price)

original_V=original_value_price(0.03161132)
print(original_V)


#0.27272727, 0.5 ,0.21090909, 0.21090909, 0, 0.52884615 ,0.96937213, 0.16559213 , price = 0.17835178
#0.18181818, 0.16666667, 0.06909091, 0.06909091, 0, 0.32264957 ,0.81699847, 0.19681032, price=0.03161132


(1, 8)
[0.03853764]
[241655.50318127]
213500.0158


In [798]:
cv_lassoCV = cross_validate(model_lasso, X_train, y_train, cv=10, scoring = "neg_mean_squared_error", return_train_score=True) 
print("--------------------------LASSO-------------------")
print("Average MSE:")
print(abs(np.average(cv_lassoCV["test_score"])))
print("\n Worst case MSE;")
print(abs(np.min(cv_lassoCV["test_score"])))
print("\n Best case MSE Linear;")
print(abs(np.max(cv_lassoCV["test_score"])))


mse =MSE(y_test,y_pred)
print("MSE-------")
print(mse)

--------------------------LASSO-------------------
Average MSE:
0.0037666119966633807

 Worst case MSE;
0.01638793484766216

 Best case MSE Linear;
0.00068754278749252
MSE-------
0.0008256504125068595


In [799]:
alpha= np.arange(0.0000001, 0.1, 0.000001)
modelo_ridgeCV = RidgeCV(alphas=alpha, fit_intercept=False).fit(X_train, y_train)

model_ridge = Ridge(alpha = modelo_ridgeCV.alpha_, normalize = False,fit_intercept=False, tol = 0.001, \
              solver ='sag', random_state = 123)

y_p=model_lr.predict(X_test)

error = y_p - y_test
print(error)
squared_error = np.dot(error, error)
mean_squared_error = squared_error / y_test.shape[0]
print(np.sqrt(mean_squared_error))

[-0.0069705  -0.03394791  0.00530209 -0.03394791 -0.00761793  0.07012449
  0.01328145  0.05684601  0.0353796  -0.00224126  0.03563547  0.04343195
  0.010541    0.00692632 -0.05039812  0.00939344 -0.0236542   0.02775519
 -0.00219527  0.01251202 -0.00831799  0.0236794   0.01838767 -0.01303515
 -0.00738492]
0.02873946682462949


In [800]:
#model_ridge.fit(X_train, y_train)
cv_ridge = cross_validate(model_ridge, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error", return_train_score=True) 
print("--------------------------RIDGE-------------------")
print("Average MSE:")
print(abs(np.average(cv_ridge["test_score"])))
print("\n Worst case MSE;")
print(abs(np.min(cv_ridge["test_score"])))
print("\n Best case MSE Linear;")
print(abs(np.max(cv_ridge["test_score"])))
print(modelo_ridgeCV.alpha_)

--------------------------RIDGE-------------------
Average MSE:
0.003654061761651059

 Worst case MSE;
0.016810649445892296

 Best case MSE Linear;
0.0007504406272923391
0.09999910000000002
